# Example for usage ARc Policy via a tool for a Strands Agent

Install requirements

In [ ]:
%pip install -r requirements.txt --upgrade

## Setup for the agent

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent
import boto3
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
import logging

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
# StrandsTelemetry().setup_console_exporter()

In [ ]:
# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ID = "<REPLACE ME>"
GUARDRAIL_ID = "<REPLACE ME>"
GUARDRAIL_VERSION = "<REPLACE ME>"
# MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
MODEL_ID = "us.amazon.nova-lite-v1:0"

In [ ]:

class NotifyOnlyGuardrailsHook(HookProvider):
    def __init__(self, guardrail_id: str, guardrail_version: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(MessageAddedEvent, self.process_message)


    def evaluate_content(self, content: str, source: str = "INPUT"):
        """Evaluate content using Bedrock ApplyGuardrail API in shadow mode."""
        print(f"[GUARDRAIL] [evaluate_content]: source: {source}, content: {content}")
        
        # User input, no need to call the guardrail in this case yet
        if source == "INPUT":
            self.input = content
            return
        
        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]

        print(f"[GUARDRAIL] [evaluate_content] Attempting to determine if the LLM output is safe.")
        
        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        print(f"[GUARDRAIL] [evaluate_content] response: {response}")

        # TODO Include re-writing the response here
        
    def process_message(self, event: MessageAddedEvent) -> None:
        """Check message and determine if it's user or assistant created."""
        print (f'[GUARDRAIL] [process_message] event: {event}')

        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        source = "INPUT"
        if event.message.get("role") == "assistant":
            source = "OUTPUT"
        
        # Call the eval function to determine if something should be done
        self.evaluate_content(content, source)

# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findgins aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

# Create agent with the guardrail-protected model
auto_policy_agent = Agent(
    system_prompt="You are a helpful assistant that will answer questions related to auto insurance policy.",
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION)],
)


In [ ]:
# Use the protected agent for conversations
response = auto_policy_agent("If I have a trailer and I get into an accident is that considered a separate vehicle?")

# TODO Do something with the response